In [18]:

PINECONE_APIKEY = ""
SUPABASE_URL = ""
SUPABASE_KEY = ""

!pip install gradio youtube_transcript_api langchain nltk sentence_transformers pinecone-client supabase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import gradio
from youtube_transcript_api import YouTubeTranscriptApi

def yt2transcript(video_id):
  print(f"\n\nid: {video_id}")

  # data looks like [{'text': 'hey friends welcome to one little coder', 'start': 0.84, 'duration': 4.38}, ...]
  data = YouTubeTranscriptApi.get_transcript(video_id)
  transcript = ' '.join([x['text'] for x in data])

  # TODO if there is no transcript (how likely is this?), run through whisper-large on hf (but 30k free characters per month)

  return transcript

#demo = gr.Interface(fn=yt2transcript, inputs=["text"], outputs=["text"])
                    #article="\n".join([f"- {k}: " + v.replace("\n"," ") for k,v in instructions.items()]))
#demo.launch(debug=True, share=True)

In [10]:
video_id = "mbn6yLcoyzo"
transcript = yt2transcript(video_id)
print(transcript)



id: mbn6yLcoyzo
hey friends welcome to one little coder if you ever wanted to build a machine learning application you would know that sometimes you have to wrap this entire machine learning application as an API so for your application you need front-end and back-end sometimes your front end could be not python for example it could be react.js it could be angularjsr it could be any JavaScript framework that you want to use while the backend being powered by Python and most of the time people actually use apis for it the APA is communicating to the front end and making calls to the server and this is how the full stack application is going to look like but traditionally if you wanted to use an API you need to host it on a server and then use something like Fast API and then get the APA so this is what we have been doing all the while but today or probably yesterday night radio announced a new feature where you can use gradu as an API this has been there for a while we probably would 

In [3]:
"""Interface for Pinecone vector stores."""
import uuid
import pinecone
from abc import ABC, abstractmethod
from typing import Any, Callable, Dict, Iterable, List, Optional

from langchain.docstore.document import Document
from langchain.embeddings.base import Embeddings
from langchain.vectorstores.base import VectorStore


class Pinecone(VectorStore):
    """Interface for vector stores."""

    def _query():
      pass

    def __init__(
        self, api_key: str, index_name: str, embedding_function: Callable
    ):
        """Initialize with necessary components."""
        try:
            import pinecone
        except ImportError:
            raise ValueError(
                "Could not import pinecone python package. "
                "Please install it with `pip install pinecone-client`."
            )
        self.embedding_function = embedding_function
        self.index_name = index_name
        #try:
        pinecone.init(
           api_key=api_key,
           environment='us-west1-gcp' # only option for for free tier
        )
        #except ValueError as e:
        #    raise ValueError(
        #        f"Your elasticsearch client string is misformatted. Got error: {e} "
         #   )
        self.client = pinecone

    def add_texts(
        self, texts: Iterable[str], metadatas: Optional[List[dict]] = None
    ) -> None:
        """Run more texts through the embeddings and add to the vectorstore."""
        index = self.client.Index(self.index_name)
        batch_size = 16 # recommended limit is 100 vectors
        for i in range(0, len(texts), batch_size):
          i_end = min(i+batch_size, len(texts))
          text_batch = texts[i:i_end]
          metadata_batch = metadatas[i:i_end] if metadatas else [{}] * (i_end-i)
          embedding_batch = self.embedding_function(text_batch) # [[0] * 768] * (i_end - i) #
          to_upsert = [
              (
                  str(uuid.uuid4()), # id that we currently don't care about
                  embedding.tolist(),
                  dict(
                      {"text": text},
                      **metadata # if 'text' in here too, it takes precendence
                  )
              ) for text, embedding, metadata in zip(text_batch, embedding_batch, metadata_batch)
          ]
          print(embedding_batch)
          print(999999)
          print(i, len(texts)/batch_size, to_upsert)
          index.upsert(vectors=to_upsert)

    def similarity_search(self, query: str, k: int = 4) -> List[Document]:
        """Return docs most similar to query."""

    @classmethod
    def from_texts(
        cls,
        texts: List[str],
        embedding: Embeddings,
        metadatas: Optional[List[dict]] = None,
        **kwargs: Any
    ) -> "VectorStore":
        """Return VectorStore initialized from texts and embeddings."""

# TODO fill out other 2 methods for Pinecone Vectore Store and ask if harrison would be open to a PR
# TODO account for mpnet's limit of 384 word pieces per chunk (is it done already?)
# DONE need to check if embeddings exist for given video id before generating embeddings
# supabase to store index (apparently can't rely on vector db to do it?) and user's curations / popular curations
# - paused after 1 week inactivity (and i believe pinecone index DELETED after some days of inactivity?!)
# - - TODO backup both pinecone and supabase daily (this should count as the activity), and make publicly accessible
# TODO user prefs data model (their curations), and curation data model


In [22]:
from langchain.text_splitter import SpacyTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

embedder = HuggingFaceEmbeddings().embed_documents
model_name = HuggingFaceEmbeddings().model_name 

from supabase import create_client, Client

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

def transcript2chunks(transcript):
  return SpacyTextSplitter().split_text(transcript)

def ingest_transcript(transcript):
  p = Pinecone(PINECONE_APIKEY, 'semantic-curations', embedder)
  chunks = transcript2chunks(transcript) 
  p.add_texts(chunks, [{'yt_video_id': video_id}] * len(chunks))

def already_ingested(yt_video_id: str):
  data = supabase.table("ingested_youtube_videos").select("*", count="estimated").eq('video_id', yt_video_id).execute()
  return data.count > 0

def ingest_video(video_id):
  if already_ingested(video_id):
    return "dupe"
  else:
    p = Pinecone(PINECONE_APIKEY, 'semantic-curations', embedder)
    transcript = yt2transcript(video_id)
    chunks = transcript2chunks(transcript)
    p.add_texts(chunks, [{'yt_video_id': video_id}] * len(chunks))
    data = supabase.table("ingested_youtube_videos").insert({"video_id": video_id, "embedding_model": model_name}).execute()
    return "ingested"

ingest_video(video_id)

'dupe'

In [ ]:

chunks = transcript2chunks(transcript)
print(len(chunks), [len(x) for x in chunks])